In [13]:
# This is only for display purposes to monitor processing easier
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

### This begins the video pre-processing

In [17]:
import os

# change this
image_instead_of_video = False

vid_file = 'sample.mp4'
img_file = 'lego.png'

unique_colors = 32


#dont need to change these
repo_path = os.path.dirname(os.getcwd())
vid_path = '/vid/'
img_path = '/img/'

if image_instead_of_video == True:
    testfile = repo_path + img_path + img_file
else:
    testfile = repo_path + vid_path + vid_file

print("TESTFILE: " + testfile)

# fyi the debug link is here: https://192.168.1.9/debug/clip.html
hue_ip = '192.168.1.9'
hue_user = '082Gdixhk1IWMX78CpodeFxBWSLbeEmO7PjpFFKy'


TESTFILE: /Users/jberry003/Documents/Dev/Projects/shine/vid/sample.mp4


In [18]:
def round_colors(m, diff_colors):
    m = np.float32(m)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 500, 1)
    ret, label, center = cv2.kmeans(m, diff_colors, None, criteria, 30, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    return center[label.flatten()]

In [19]:
import cv2
import numpy as np

def extract_dominant(pic_in, clusters):
    
    # normalize array to 0-1 instead of 0-255
    pic_norm = np.array(pic_in).astype('float32')/255
    
    # convert to LAB
    pic_lab = cv2.cvtColor(pic_norm, cv2.COLOR_BGR2LAB)
    
    # reshape - make it long and 2D
    pic_reshape = np.float32(pic_lab.reshape(-1,3))
    
    # cluster
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5000, 5)
    flags = cv2.KMEANS_PP_CENTERS

    _, labels, palette = cv2.kmeans(pic_reshape, clusters, None, criteria, 30, flags)
    _, counts = np.unique(labels, return_counts=True)

    # most dominant cluster
    biggest_cluster = palette[np.argmax(counts)]
    
    # reshape to 3 channels
    biggest_cluster = np.float32(biggest_cluster.reshape(1,1,3))

    # convert back to BGR
    biggest_cluster = cv2.cvtColor(biggest_cluster, cv2.COLOR_LAB2BGR)
    
    # de-normalize back to bits
    biggest_cluster *= [255,255,255]
    #print('BIGGEST: ', biggest_cluster[0][0])
    return np.rint(biggest_cluster[0][0])

In [23]:
import datetime

def parse_to_array(vid, n):
    
    k_per_frame = 5
    
    update_progress(0)
    
    cap = cv2.VideoCapture(vid)
    fps = cap.get(cv2.CAP_PROP_FPS)
    tot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    f = 0
    result = []
    times = []
    while(True):
        grabbed, frame = cap.read()
        if not grabbed:
            break
        f+=1
        sec = float(f)/fps
        if (f % n == 0 and f > -1):
            s = datetime.datetime.now()
            result.append(extract_dominant(frame, k_per_frame)[:3])
            times.append(sec)
            e = datetime.datetime.now()
            t = e - s
            #print(str(t.total_seconds() * 1000))
            update_progress(f / tot)
    cap.release()
    return result, times

In [24]:
from rgbxy import Converter
from rgbxy import GamutC # or GamutA, GamutB, GamutC

converter = Converter(GamutC)

def change_color(r,g,b):
    bri.lights[6].state(on=True, xy=converter.rgb_to_xy(r,g,b))
    
def light_off():
    bri.lights[6].state(on=False)

In [ ]:
# take video file and take every nth frame, grabbing dominant color from each
training = parse_to_array(testfile, 46)

Progress: [###-----------------] 17.2%


In [ ]:
result = round_colors(training[0], unique_colors)

In [ ]:
import pandas as pd
df = pd.DataFrame({'t':np.asarray(training[1]),'r':result[:,0],'g':result[:,1],'b':result[:,2]})

In [ ]:
df['helper'] = df['r'].astype(str) + "," + df['g'].astype(str) + "," + df['b'].astype(str)
df['counter'] = df.groupby((df['helper'] != df['helper'].shift(1)).cumsum()).cumcount()+1
df.head(5)

### This begins the Hue interaction - play video separately

In [ ]:
# loop through dataframe and show colors
import time

light_off()

print("     in 5,...")
time.sleep(5)
print("GO!")
time.sleep(1)


start = time.time()
i = 1
while i < df['t'].max():
    now = time.time()
    diff = now - start
    
    row = df.iloc[(df['t'] - diff).abs().argsort()[:1]]
    r = row[['r']].values[0][0]
    g = row[['g']].values[0][0]
    b = row[['b']].values[0][0]
    
    if i == 1:
        newxy = converter.rgb_to_xy(r,g,b)
        oldxy = newxy
        change_color(r,g,b)
    else:
        oldxy = newxy
        newxy = converter.rgb_to_xy(r,g,b)
    
    if oldxy != newxy:
        print("CHANGED!  (" + str(r) + "," + str(g) + "," + str(b) + ")")
        change_color(r,g,b)
        time.sleep(2)
        
    i+=1
    time.sleep(1.0 - ((time.time() - start) % 1.0))
    

light_off()